In [91]:
import pandas as pd

# Load the CSV file
file_path = "Copy of 2023 March Madness Optimal Bracket - ESPN.csv"
df = pd.read_csv(file_path)

# Extract relevant columns and clean the data
rounds = ['R2', 'R3', 'R4', 'R5', 'R6', 'R7']


cleaned_data = []

# Iterate through each round column and extract team names and percentages
for round_name in rounds:
    for entry in df[round_name][1:]: 
        if pd.notna(entry) and '-' in entry and '/' not in entry:
            try:
                team, percent = entry.rsplit('-', 1)  # Split team name and percentage
                percent = float(percent.replace('%', ''))  # Convert to float
                cleaned_data.append({'Team': team.strip(), 'Round': round_name, 'ESPN_Percentage': percent})
            except ValueError:
                continue  # Skip any malformed entries


espn_df = pd.DataFrame(cleaned_data)

# Extract seed/ranking and clean team names
espn_df['Team'] = espn_df['Team'].str.replace(r'^[0-9]+', '', regex=True).str.strip()

# Pivot the data for better readability
espn_table = espn_df.pivot(index='Team', columns='Round', values='ESPN_Percentage').fillna(0).reset_index()

pd.set_option('display.max_columns', None)

print (espn_table)

Round           Team    R2    R3    R4    R5    R6    R7
0            Alabama  96.6  88.4  76.4  56.5  40.3  25.4
1            Arizona  93.4  80.4  54.7  18.6  10.2   4.4
2           Arkansas  58.1   7.9   4.2   1.7   0.7   0.4
3             Auburn  47.4   8.2   3.6   1.4   0.6   0.3
4             Baylor  90.3  62.9  24.9   9.2   4.2   1.4
5           Boise St  35.3   5.1   1.6   0.6   0.3   0.1
6         Charleston  30.3  10.5   1.5   0.6   0.3   0.1
7            Colgate   7.0   3.0   1.5   0.5   0.2   0.1
8          Creighton  67.9  24.5   9.0   2.4   0.8   0.3
9              Drake  22.9   7.6   1.7   0.6   0.2   0.1
10              Duke  81.9  46.1  22.6  14.2   6.0   3.2
11               FAU  27.9   4.3   1.7   0.7   0.3   0.1
12            Furman  15.7   5.3   1.1   0.4   0.2   0.1
13           Gonzaga  92.7  71.5  33.5  15.8   8.5   4.1
14      Grand Canyon   7.3   3.1   1.1   0.4   0.2   0.1
15           Houston  95.2  81.8  64.6  43.2  28.0  10.7
16            Howard   4.7   2.

In [92]:
# Load the FiveThirtyEight table
fte_table = pd.read_csv("FiveThirtyEight Data - 538.csv")

# Extract and Rename probability columns
probability_columns = ["team_name", "team_seed", "team_region", "rd2_win", "rd3_win", "rd4_win", "rd5_win", "rd6_win", "rd7_win",]
fte_table = fte_table[probability_columns].copy()

fte_table.rename(columns={
    'rd2_win': 'R2', 'rd3_win': 'R3', 'rd4_win': 'R4',
    'rd5_win': 'R5', 'rd6_win': 'R6', 'rd7_win': 'R7',
    'team_name': 'Team', 'team_seed': 'Seed', 'team_region': 'Region'
}, inplace=True)

# Remove the percentage signs from the specified columns
fte_table[['R2', 'R3', 'R4', 'R5', 'R6', 'R7']] = fte_table[['R2', 'R3', 'R4', 'R5', 'R6', 'R7']].replace('%', '', regex=True)

# Convert the columns to numeric values
fte_table[['R2', 'R3', 'R4', 'R5', 'R6', 'R7']] = fte_table[['R2', 'R3', 'R4', 'R5', 'R6', 'R7']].apply(pd.to_numeric)

print (fte_table)

                        Team Seed   Region     R2     R3     R4     R5     R6  \
0                    Houston    1  Midwest  97.39  76.20  61.66  43.71  33.86   
1                    Alabama    1    South  98.52  81.12  64.02  43.68  28.93   
2                      Texas    2  Midwest  92.16  63.82  44.72  21.02  13.30   
3                     Purdue    1     East  98.52  71.81  42.24  26.36  12.75   
4                     Kansas    1     West  97.82  65.63  38.99  20.27   8.75   
..                       ...  ...      ...    ...    ...    ...    ...    ...   
63  Texas A&M-Corpus Christi  16a    South   0.87   0.09   0.01   0.00   0.00   
64  Southeast Missouri State  16b    South   0.60   0.05   0.01   0.00   0.00   
65                    Howard   16     West   2.18   0.14   0.01   0.00   0.00   
66            Texas Southern  16a     East   0.91   0.06   0.00   0.00   0.00   
67       Fairleigh Dickinson  16b     East   0.57   0.03   0.00   0.00   0.00   

       R7  
0   24.92  
1  

In [93]:
# Mapping of alternative team names to standardized names
team_name_mapping = {
    "College of Charleston": "Charleston",
    "Miami (FL)": "Miami",
    "Michigan State": "Michigan St",
    "Kansas State": "Kansas St",
    "Boise State": "Boise St",
    "Kennesaw State": "Kennesaw St",
    "Louisiana-Lafayette": "Louisiana",
    "North Carolina State": "NC State",
    "North Carolina-Asheville": "UNC Asheville",
    "Northern Kentucky": "N Kentucky",
    "Saint Mary's (CA)": "Saint Mary's",
    "San Diego State": "San Diego St",
    "Southern California": "USC",
    "Texas Christian": "TCU",
    "UC-Santa Barbara": "UCSB",
    "Virginia Commonwealth": "VCU",
    "Connecticut": "UConn",
    "Florida Atlantic": "FAU",
    "Montana State": "Montana St", 
}

# Apply the mapping to both tables before merging
espn_table["Team"] = espn_table["Team"].replace(team_name_mapping)
fte_table["Team"] = fte_table["Team"].replace(team_name_mapping)

fte_table = fte_table.rename(columns={'Seed': 'Seed_538'})
fte_table = fte_table.rename(columns={'Region': 'Region_538'})

# Merge ESPN and FiveThirtyEight tables on Team
merged_table = pd.merge(espn_table, fte_table, on='Team', how='outer', suffixes=("_ESPN", "_538"))


if 'Region_538' in merged_table.columns:
    merged_table['Region'] = merged_table['Region_538'].fillna("Unknown").astype(str)
else:
    print("Warning: Region_538 column not found in merged_table")

# Ensure a single Seed column
# Use the Seed from fte_table if available, otherwise use 0 as a placeholder
merged_table['Seed'] = merged_table['Seed_538'].fillna(0).astype(str)
merged_table['Region'] = merged_table['Region_538'].fillna(0).astype(str)

merged_table = merged_table[['Team', 'Seed', 'Region'] + [f'{r}_ESPN' for r in rounds] + [f'{r}_538' for r in rounds]]

# Fill NaN values with 0 for readability
merged_table.fillna(0, inplace=True)

print(merged_table)

                        Team Seed   Region  R2_ESPN  R3_ESPN  R4_ESPN  \
0                    Alabama    1    South     96.6     88.4     76.4   
1                    Arizona    2    South     93.4     80.4     54.7   
2                   Arkansas    8     West     58.1      7.9      4.2   
3                     Auburn    9  Midwest     47.4      8.2      3.6   
4                     Baylor    3    South     90.3     62.9     24.9   
..                       ...  ...      ...      ...      ...      ...   
63                    Nevada  11b     West      0.0      0.0      0.0   
64  Texas A&M-Corpus Christi  16a    South      0.0      0.0      0.0   
65  Southeast Missouri State  16b    South      0.0      0.0      0.0   
66            Texas Southern  16a     East      0.0      0.0      0.0   
67       Fairleigh Dickinson  16b     East      0.0      0.0      0.0   

    R5_ESPN  R6_ESPN  R7_ESPN  R2_538  R3_538  R4_538  R5_538  R6_538  R7_538  
0      56.5     40.3     25.4   98.52   81.

In [94]:
leverage = pd.DataFrame()
leverage['Team'] = merged_table['Team']
leverage['Seed'] = merged_table['Seed']
leverage['Region'] = merged_table['Region']
leverage['R64_Lev'] = (merged_table['R2_538'] - merged_table['R2_ESPN']) * merged_table['R2_538']
leverage['R32_Lev'] = (merged_table['R3_538'] - merged_table['R3_ESPN']) * merged_table['R3_538']
leverage['S16_Lev'] = (merged_table['R4_538'] - merged_table['R4_ESPN']) * merged_table['R4_538']
leverage['E8_Lev'] = (merged_table['R5_538'] - merged_table['R5_ESPN']) * merged_table['R5_538']
leverage['F4_Lev'] = (merged_table['R6_538'] - merged_table['R6_ESPN']) * merged_table['R6_538']
leverage['NCG_Lev'] = (merged_table['R7_538'] - merged_table['R7_ESPN']) * merged_table['R7_538']

# Display the leverage table
print(leverage)

                        Team Seed   Region   R64_Lev   R32_Lev   S16_Lev  \
0                    Alabama    1    South  189.1584 -590.5536 -792.5676   
1                    Arizona    2    South   65.8700 -860.3399 -687.3384   
2                   Arkansas    8     West -334.0461  180.3975   38.1004   
3                     Auburn    9  Midwest  609.3184   93.8604   38.7504   
4                     Baylor    3    South -124.4600 -801.4125  -18.3464   
..                       ...  ...      ...       ...       ...       ...   
63                    Nevada  11b     West  190.1641   11.9025    1.5129   
64  Texas A&M-Corpus Christi  16a    South    0.7569    0.0081    0.0001   
65  Southeast Missouri State  16b    South    0.3600    0.0025    0.0001   
66            Texas Southern  16a     East    0.8281    0.0036    0.0000   
67       Fairleigh Dickinson  16b     East    0.3249    0.0009    0.0000   

      E8_Lev    F4_Lev   NCG_Lev  
0  -559.9776 -328.9341 -157.4875  
1   -54.6804  -14

In [95]:
# Sort by NCG_Lev from greatest to least
leverage = leverage.sort_values(by='NCG_Lev', ascending=False)

# Clean 'Seed' column by removing non-numeric characters
leverage['Seed'] = leverage['Seed'].str.extract('(\d+)').astype(int)

original_leverage = leverage.copy()

print(leverage)

         Team  Seed   Region   R64_Lev    R32_Lev    S16_Lev    E8_Lev  \
15    Houston     1  Midwest  213.2841  -426.7200  -181.2804   22.2921   
47      Texas     2  Midwest  -77.4144  -681.5976  -638.6016 -132.0056   
8   Creighton     6    South  828.5376   970.1784   442.8604  101.1769   
51      UConn     4     West  -74.5536   213.4881   510.1919  114.1416   
4      Baylor     3    South -124.4600  -801.4125   -18.3464    5.7761   
..        ...   ...      ...       ...        ...        ...       ...   
1     Arizona     2    South   65.8700  -860.3399  -687.3384  -54.6804   
49       UCLA     2     West  168.0096  -897.1159  -464.7529   17.6375   
42     Purdue     1     East  238.4184  -516.3139  -429.1584 -327.9184   
22     Kansas     1     West  246.5064 -1218.7491 -1158.3929 -507.3581   
0     Alabama     1    South  189.1584  -590.5536  -792.5676 -559.9776   

      F4_Lev   NCG_Lev  
15  198.4196  354.3624  
47    2.6600   14.0096  
8    32.5584    5.2216  
51   14.582

In [96]:
champion = leverage.loc[leverage['NCG_Lev'].idxmax(), 'Team']
leverage = leverage[leverage['Team'] != champion]

print(f"Champion: {champion}")

Champion: Houston


In [97]:
leverage = leverage.sort_values(by='F4_Lev', ascending=False)

champion_region = original_leverage.loc[original_leverage['Team'] == champion, 'Region'].values[0]  # Get region directly
opposite_regions = {'South', 'East'} if champion_region in {'Midwest', 'West'} else {'Midwest', 'West'}
finalist_candidates = leverage[leverage['Region'].isin(opposite_regions)]

if not finalist_candidates.empty:
    finalist = finalist_candidates.loc[finalist_candidates['F4_Lev'].idxmax(), 'Team']
    leverage = leverage[leverage['Team'] != finalist]
else:
    finalist = None  # Edge case if no valid finalist exists

finals = set()
finals.add(champion)
finals.add(finalist)

print(f"Finals: {finals}")

Finals: {'Creighton', 'Houston'}


In [98]:
final_four = finals.copy()

leverage = leverage.sort_values(by='E8_Lev', ascending=False)
while len(final_four) < 4:
    for _, row in leverage.iterrows():
        # Get the seeds of all eliminated teams from the original data
        eliminated_reg = set(original_leverage.loc[original_leverage['Team'].isin(list(final_four)), 'Region'])
        
        # Only add the candidate if its seed is not already in the eliminated set
        if row['Region'] not in eliminated_reg:
            final_four.add(row['Team'])
            leverage = leverage[leverage['Team'] != row['Team']]
            break


print(f"Final Four: {final_four}")

Final Four: {'Creighton', 'Gonzaga', 'Kentucky', 'Houston'}


In [99]:
elite_eight = final_four.copy()
region_counts = {region: 0 for region in leverage['Region'].unique()}  # Track teams per region in Elite Eight

# Pick remaining Elite Eight teams
leverage = leverage.sort_values(by='S16_Lev', ascending=False)
while len(elite_eight) < 8:
    for _, row in leverage.iterrows():
        team_region = row['Region']
        region_teams = [
            team for team in elite_eight 
            if (not original_leverage[original_leverage['Team'] == team].empty and 
                original_leverage[original_leverage['Team'] == team]['Region'].values[0] == team_region)
        ]
        if len(region_teams) < 2:
            #Checks for conflicting seeds
            region_seeds = [
                original_leverage[original_leverage['Team'] == team]['Seed'].values[0]
                for team in region_teams
                if not original_leverage[original_leverage['Team'] == team].empty
            ]
            group1 = {1, 16, 8, 9, 5, 12, 4, 13}
            group2 = {2, 15, 7, 10, 3, 14, 6, 11}
            conflicting_seeds = group1 if row['Seed'] in group1 else group2
            if not any(seed in conflicting_seeds for seed in region_seeds):
                elite_eight.add(row['Team'])
                region_counts[team_region] += 1
                leverage = leverage[leverage['Team'] != row['Team']]
                break

print(f"Elite Eight: {elite_eight}")

Elite Eight: {'Kentucky', 'San Diego St', 'Creighton', 'UConn', 'Gonzaga', 'Tennessee', 'Iowa State', 'Houston'}


In [100]:
leverage = leverage.sort_values(by='R32_Lev', ascending=False)

sweet_sixteen = elite_eight.copy()

region_seed_groups = {}  # Maps each region to the seed groups already selected in Elite Eight

for team in elite_eight:
    team_region = original_leverage.loc[original_leverage['Team'] == team, 'Region'].values[0]
    team_seed = original_leverage.loc[original_leverage['Team'] == team, 'Seed'].values[0]
    # Determine seed group
    if team_seed in {1, 16, 8, 9}:
        seed_group = 1
    elif team_seed in {5, 12, 4, 13}:
        seed_group = 2
    elif team_seed in {6, 11, 3, 14}:
        seed_group = 3
    elif team_seed in {7, 10, 2, 15}:
        seed_group = 4
    else:
        print(f"Unexpected seed: {team_seed} for team {row['Team']}")  # Debugging statement
        continue  

    if team_region not in region_seed_groups:
        region_seed_groups[team_region] = set()
    region_seed_groups[team_region].add(seed_group)
    

while len(sweet_sixteen) < 16:
    candidate_found = False
    for _, row in leverage.iterrows():
        team_region = row['Region']
        team_seed = int(row['Seed'])
        region_teams = [
            team for team in sweet_sixteen
            if original_leverage.loc[original_leverage['Team'] == team, 'Region'].values[0] == team_region
        ]
        
        # Ensure no more than 4 teams per region
        if len(region_teams) < 4:
            # Determine seed group
            if team_seed in {1, 16, 8, 9}:
                candidate_group = 1
            elif team_seed in {5, 12, 4, 13}:
                candidate_group = 2
            elif team_seed in {6, 11, 3, 14}:
                candidate_group = 3
            elif team_seed in {7, 10, 2, 15}:
                candidate_group = 4
            else:
                continue  # Ignore unexpected seeds

            # Check if candidate's seed group is already filled in this region
            if team_region in region_seed_groups and candidate_group in region_seed_groups[team_region]:
                continue  # Skip candidate if seed group is full
            
            # Add the candidate to Sweet Sixteen
            sweet_sixteen.add(row['Team'])
            leverage = leverage[leverage['Team'] != row['Team']]
            
            # Update region seed group tracking
            if team_region not in region_seed_groups:
                region_seed_groups[team_region] = set()
            region_seed_groups[team_region].add(candidate_group)

            candidate_found = True
            break

print(f"Sweet Sixteen: {sweet_sixteen}")

Sweet Sixteen: {'Arkansas', 'Kentucky', 'San Diego St', 'Creighton', 'Michigan St', 'UConn', 'Texas A&M', 'Boise St', 'Drake', 'Maryland', 'Utah State', 'Memphis', 'Gonzaga', 'Tennessee', 'Iowa State', 'Houston'}


In [101]:
round_of_32 = sweet_sixteen.copy() # The teams moving into the Round of 32

leverage = leverage.sort_values(by='R64_Lev', ascending=False)

region_seed_groups = {}  # Tracks selected seed groups per region

for team in sweet_sixteen:
    team_region = original_leverage.loc[original_leverage['Team'] == team, 'Region'].values[0]
    team_seed = original_leverage.loc[original_leverage['Team'] == team, 'Seed'].values[0]
    
    # Assign seed groups based on updated pairs
    if team_seed in {1, 16}:
        seed_group = 1
    elif team_seed in {8, 9}:
        seed_group = 2
    elif team_seed in {2, 15}:
        seed_group = 3
    elif team_seed in {3, 14}:
        seed_group = 4
    elif team_seed in {4, 13}:
        seed_group = 5
    elif team_seed in {5, 12}:
        seed_group = 6
    elif team_seed in {6, 11}:
        seed_group = 7
    elif team_seed in {7, 10}:
        seed_group = 8
    else:
        print(f"Unexpected seed: {team_seed} for team {team}")
        continue  
    
    if team_region not in region_seed_groups:
        region_seed_groups[team_region] = set()
    region_seed_groups[team_region].add(seed_group)
    
while len(round_of_32) < 32:
    candidate_found = False
    for _, row in leverage.iterrows():
        team_region = row['Region']
        team_seed = int(row['Seed'])
        region_teams = [
            team for team in round_of_32
            if original_leverage.loc[original_leverage['Team'] == team, 'Region'].values[0] == team_region
        ]
        
        # Ensure no more than 8 teams per region
        if len(region_teams) < 8:
            # Determine seed group
            if team_seed in {1, 16}:
                candidate_group = 1
            elif team_seed in {8, 9}:
                candidate_group = 2
            elif team_seed in {2, 15}:
                candidate_group = 3
            elif team_seed in {3, 14}:
                candidate_group = 4
            elif team_seed in {4, 13}:
                candidate_group = 5
            elif team_seed in {5, 12}:
                candidate_group = 6
            elif team_seed in {6, 11}:
                candidate_group = 7
            elif team_seed in {7, 10}:
                candidate_group = 8
            else:
                continue  # Ignore unexpected seeds
            
            # Check if seed group is already filled in this region
            if team_region in region_seed_groups and candidate_group in region_seed_groups[team_region]:
                continue  # Skip candidate if seed group is full
            
            # Add candidate to Round of 32
            round_of_32.add(row['Team'])
            leverage = leverage[leverage['Team'] != row['Team']]
            
            # Update region seed group tracking
            if team_region not in region_seed_groups:
                region_seed_groups[team_region] = set()
            region_seed_groups[team_region].add(candidate_group)
            
            candidate_found = True
            break

print(f"Round of 32: {round_of_32}")

Round of 32: {'Colgate', 'Arkansas', 'Kansas', 'Texas A&M', 'Arizona State', 'Maryland', 'UConn', 'Duke', 'Boise St', 'Kennesaw St', 'UCSB', 'UCLA', 'San Diego St', 'Arizona', 'Memphis', 'Iowa State', 'Auburn', 'Alabama', 'Kentucky', 'Creighton', 'Michigan St', 'Vermont', 'Montana St', 'Drake', 'Kent State', 'Utah State', 'VCU', 'Furman', 'Purdue', 'Gonzaga', 'Tennessee', 'Houston'}


In [102]:

print("\n=== March Madness Bracket ===")

print("\n🏆 Champion:")
print(champion)

print("\n🏅 Finals:")
print(", ".join(sorted(finals)))

print("\n🔥 Final Four:")
print(", ".join(sorted(final_four)))

print("\n💪 Elite Eight:")
print(", ".join(sorted(elite_eight)))

print("\n🔥 Sweet Sixteen:")
print(", ".join(sorted(sweet_sixteen)))

print("\n🏀 Round of 32:")
print(", ".join(sorted(round_of_32)))





=== March Madness Bracket ===

🏆 Champion:
Houston

🏅 Finals:
Creighton, Houston

🔥 Final Four:
Creighton, Gonzaga, Houston, Kentucky

💪 Elite Eight:
Creighton, Gonzaga, Houston, Iowa State, Kentucky, San Diego St, Tennessee, UConn

🔥 Sweet Sixteen:
Arkansas, Boise St, Creighton, Drake, Gonzaga, Houston, Iowa State, Kentucky, Maryland, Memphis, Michigan St, San Diego St, Tennessee, Texas A&M, UConn, Utah State

🏀 Round of 32:
Alabama, Arizona, Arizona State, Arkansas, Auburn, Boise St, Colgate, Creighton, Drake, Duke, Furman, Gonzaga, Houston, Iowa State, Kansas, Kennesaw St, Kent State, Kentucky, Maryland, Memphis, Michigan St, Montana St, Purdue, San Diego St, Tennessee, Texas A&M, UCLA, UCSB, UConn, Utah State, VCU, Vermont
